In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
dataset= pd.read_csv("/Users/rishika/Desktop/berlin-airbnb-data/listings_summary.csv")


In [3]:
# Missing values
dataset.amenities= dataset.amenities.str.count(',')
dataset.bathrooms= dataset.bathrooms.fillna(1)
dataset.bedrooms= dataset.bedrooms.fillna(1)
dataset.beds= dataset.beds.fillna(1)


In [4]:
# Calculating distance using latitude and longitude
from geopy.distance import great_circle
def distance_from_berlin(lat,lon):
    center=(52.5027778, 13.404166666666667)
    record=(lat,lon)
    return great_circle(center,record).km
dataset['distance'] = dataset.apply(lambda x: distance_from_berlin(x.latitude, x.longitude), axis=1)

features=["distance","accommodates","bathrooms","bedrooms","beds","minimum_nights","amenities","guests_included","number_of_reviews","availability_365"]

In [5]:
(dataset[features]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22552 entries, 0 to 22551
Data columns (total 10 columns):
distance             22552 non-null float64
accommodates         22552 non-null int64
bathrooms            22552 non-null float64
bedrooms             22552 non-null float64
beds                 22552 non-null float64
minimum_nights       22552 non-null int64
amenities            22552 non-null int64
guests_included      22552 non-null int64
number_of_reviews    22552 non-null int64
availability_365     22552 non-null int64
dtypes: float64(4), int64(6)
memory usage: 1.7 MB


In [6]:
(dataset[features]).head(1)

,distance,accommodates,bathrooms,bedrooms,beds,minimum_nights,amenities,guests_included,number_of_reviews,availability_365
0,3.533182,3,1.0,1.0,2.0,4,28,1,118,141


In [7]:
# Categorical variables
dataset['room_type'] = pd.Categorical(dataset['room_type'])
# dataset['property_type'] = pd.Categorical(dataset['room_type'])
Dummies1 = pd.get_dummies(dataset['room_type'], prefix = 'category')
# Dummies2 = pd.get_dummies(dataset['property_type'], prefix = 'category')
# dataset2= pd.concat([Dummies1,Dummies2], axis=1)
X= pd.concat([Dummies1,dataset[features]], axis=1)

In [8]:
dataset.price =  list(map(lambda x: float(str(x).replace(',','').replace('$','')),dataset.price)) 
y= dataset["price"]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test=train_test_split(X,y,test_size= .25) 

In [10]:
# normalize data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

/Users/rishika/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/rishika/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/rishika/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  """


In [11]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
from sklearn.model_selection import GridSearchCV
param_grid = [{'max_depth': np.arange(1, 10),
              'min_samples_leaf': [1, 5, 10, 20, 50, 100],
              'min_weight_fraction_leaf': [0.0,0.1,0.3,0.5],
              'random_state':[1,3,4,7]}]
gridCV = GridSearchCV(estimator=regressor, param_grid=param_grid,cv=10)
gridCV = gridCV.fit(X_train, y_train)
print(gridCV.best_score_)
print(gridCV.best_params_)

0.02041028259146075
{'max_depth': 4, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.1, 'random_state': 1}


/Users/rishika/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [12]:
md = gridCV.best_params_["max_depth"]
msl  = gridCV.best_params_["min_samples_leaf"]
mwfl = gridCV.best_params_["min_weight_fraction_leaf"]
rs = gridCV.best_params_["random_state"]

In [13]:
regr_tree = DecisionTreeRegressor(max_depth=md,min_samples_leaf=msl,min_weight_fraction_leaf=mwfl,random_state=rs)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [14]:
#Measure error
from sklearn.metrics import mean_squared_error, r2_score
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {round(RMSE, 4)}")

r2 = r2_score(y_test, y_pred)
print(f"r2: {round(r2, 4)}")

RMSE: 234.8403
r2: -0.648
